In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
import os
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import func

In [2]:
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///../data/new_olympics.db'
db = SQLAlchemy(app)



C:\Users\norfe\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
sqlite3.connect('../data/olympics_raw.sqlite')
conn = sqlite3.connect('../data/new_olympics.db')

In [4]:
pd.read_sql('SELECT * FROM olympics_raw LIMIT 1', conn)

,id,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal,country_id
0,1,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold,1


In [6]:
 # Create engine using the `demographics.sqlite` database file
Base = automap_base()    
engine = create_engine("sqlite:///../data/new_olympics.db")
Base.prepare(engine, reflect=True)

In [7]:
Base.classes.keys()

['country_ref', 'olympics_raw']

In [8]:
Base.metadata

MetaData(bind=None)

In [9]:
class olympic_raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer)
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))

In [10]:
class raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer, db.ForeignKey('country_ref.id'))
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))
    def __repr__(self):
        return '<raw %r>' % (self.id)

In [11]:
class country_ref(db.Model):
    __tablename__ = 'country_ref' 
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    country_name = db.Column(db.String(64))
    code =db.Column(db.String(64))
    flag_source = db.Column(db.String(64))
    flag_image = db.Column(db.String(64))
    def __repr__(self):
        return '<country_ref %r>' % (self.id)


In [12]:
countries = Base.classes.country_ref
raw = Ba

NameError: name 'Ba' is not defined

In [28]:
cr = db.session.query(country_ref.id, country_ref.code, country_ref.country_name,country_ref.flag_image).all()


AttributeError: type object 'country_ref' has no attribute 'medal'

In [30]:
count = db.session.query(raw.country_id,raw.Medal, func.count(raw.id)).group_by(raw.country_id,raw.Medal).all()
count

[(1, 'Bronze', 345),
 (1, 'Gold', 400),
 (1, 'Silver', 308),
 (2, 'Bronze', 44),
 (2, 'Gold', 21),
 (2, 'Silver', 81),
 (3, 'Bronze', 49),
 (3, 'Gold', 34),
 (3, 'Silver', 62),
 (4, 'Bronze', 1052),
 (4, 'Gold', 2088),
 (4, 'Silver', 1195),
 (5, 'Bronze', 454),
 (5, 'Gold', 407),
 (5, 'Silver', 350),
 (6, 'Bronze', 505),
 (6, 'Gold', 498),
 (6, 'Silver', 591),
 (7, 'Bronze', 475),
 (7, 'Gold', 378),
 (7, 'Silver', 461),
 (8, 'Bronze', 413),
 (8, 'Gold', 293),
 (8, 'Silver', 369),
 (9, 'Bronze', 152),
 (9, 'Gold', 147),
 (9, 'Silver', 192),
 (10, 'Bronze', 138),
 (10, 'Gold', 73),
 (10, 'Silver', 165),
 (11, 'Bronze', 10),
 (11, 'Gold', 23),
 (11, 'Silver', 15),
 (12, 'Bronze', 320),
 (12, 'Gold', 212),
 (12, 'Silver', 250),
 (13, 'Bronze', 150),
 (13, 'Gold', 91),
 (13, 'Silver', 167),
 (14, 'Bronze', 34),
 (14, 'Gold', 128),
 (14, 'Silver', 16),
 (15, 'Bronze', 227),
 (15, 'Gold', 154),
 (15, 'Silver', 211),
 (16, 'Bronze', 6),
 (16, 'Silver', 1),
 (17, 'Bronze', 325),
 (17, 'Gold', 3

In [34]:
subq = (db.session.query(raw.country_id,raw.Medal,func.count(raw.id)).group_by(raw.country_id,raw.Medal)).filter(raw.Medal='Gold').subquery()

SyntaxError: keyword can't be an expression (<ipython-input-34-f055d178dfeb>, line 1)

In [33]:
db.session.query(subq).all()

[(1, 'Bronze', 345),
 (1, 'Gold', 400),
 (1, 'Silver', 308),
 (2, 'Bronze', 44),
 (2, 'Gold', 21),
 (2, 'Silver', 81),
 (3, 'Bronze', 49),
 (3, 'Gold', 34),
 (3, 'Silver', 62),
 (4, 'Bronze', 1052),
 (4, 'Gold', 2088),
 (4, 'Silver', 1195),
 (5, 'Bronze', 454),
 (5, 'Gold', 407),
 (5, 'Silver', 350),
 (6, 'Bronze', 505),
 (6, 'Gold', 498),
 (6, 'Silver', 591),
 (7, 'Bronze', 475),
 (7, 'Gold', 378),
 (7, 'Silver', 461),
 (8, 'Bronze', 413),
 (8, 'Gold', 293),
 (8, 'Silver', 369),
 (9, 'Bronze', 152),
 (9, 'Gold', 147),
 (9, 'Silver', 192),
 (10, 'Bronze', 138),
 (10, 'Gold', 73),
 (10, 'Silver', 165),
 (11, 'Bronze', 10),
 (11, 'Gold', 23),
 (11, 'Silver', 15),
 (12, 'Bronze', 320),
 (12, 'Gold', 212),
 (12, 'Silver', 250),
 (13, 'Bronze', 150),
 (13, 'Gold', 91),
 (13, 'Silver', 167),
 (14, 'Bronze', 34),
 (14, 'Gold', 128),
 (14, 'Silver', 16),
 (15, 'Bronze', 227),
 (15, 'Gold', 154),
 (15, 'Silver', 211),
 (16, 'Bronze', 6),
 (16, 'Silver', 1),
 (17, 'Bronze', 325),
 (17, 'Gold', 3

In [32]:
db.session.query(country_ref.country_name, country_ref.code, subq).join(subq).all()

[('Hungary', 'HUN', 1, 'Bronze', 345),
 ('Hungary', 'HUN', 1, 'Gold', 400),
 ('Hungary', 'HUN', 1, 'Silver', 308),
 ('Austria', 'AUT', 2, 'Bronze', 44),
 ('Austria', 'AUT', 2, 'Gold', 21),
 ('Austria', 'AUT', 2, 'Silver', 81),
 ('Greece', 'GRE', 3, 'Bronze', 49),
 ('Greece', 'GRE', 3, 'Gold', 34),
 ('Greece', 'GRE', 3, 'Silver', 62),
 ('United States', 'USA', 4, 'Bronze', 1052),
 ('United States', 'USA', 4, 'Gold', 2088),
 ('United States', 'USA', 4, 'Silver', 1195),
 ('Germany', 'GER', 5, 'Bronze', 454),
 ('Germany', 'GER', 5, 'Gold', 407),
 ('Germany', 'GER', 5, 'Silver', 350),
 ('United Kingdom', 'GBR', 6, 'Bronze', 505),
 ('United Kingdom', 'GBR', 6, 'Gold', 498),
 ('United Kingdom', 'GBR', 6, 'Silver', 591),
 ('France', 'FRA', 7, 'Bronze', 475),
 ('France', 'FRA', 7, 'Gold', 378),
 ('France', 'FRA', 7, 'Silver', 461),
 ('Australia', 'AUS', 8, 'Bronze', 413),
 ('Australia', 'AUS', 8, 'Gold', 293),
 ('Australia', 'AUS', 8, 'Silver', 369),
 ('Denmark', 'DEN', 9, 'Bronze', 152),
 ('De

In [ ]:
db.session.query(country_ref,count).filter(country_ref.id == count.country_id).all()

In [36]:
results=db.session.query(country_ref.country_name, country_ref.code, subq).join(subq).all()
results

[('Hungary', 'HUN', 1, 'Bronze', 345),
 ('Hungary', 'HUN', 1, 'Gold', 400),
 ('Hungary', 'HUN', 1, 'Silver', 308),
 ('Austria', 'AUT', 2, 'Bronze', 44),
 ('Austria', 'AUT', 2, 'Gold', 21),
 ('Austria', 'AUT', 2, 'Silver', 81),
 ('Greece', 'GRE', 3, 'Bronze', 49),
 ('Greece', 'GRE', 3, 'Gold', 34),
 ('Greece', 'GRE', 3, 'Silver', 62),
 ('United States', 'USA', 4, 'Bronze', 1052),
 ('United States', 'USA', 4, 'Gold', 2088),
 ('United States', 'USA', 4, 'Silver', 1195),
 ('Germany', 'GER', 5, 'Bronze', 454),
 ('Germany', 'GER', 5, 'Gold', 407),
 ('Germany', 'GER', 5, 'Silver', 350),
 ('United Kingdom', 'GBR', 6, 'Bronze', 505),
 ('United Kingdom', 'GBR', 6, 'Gold', 498),
 ('United Kingdom', 'GBR', 6, 'Silver', 591),
 ('France', 'FRA', 7, 'Bronze', 475),
 ('France', 'FRA', 7, 'Gold', 378),
 ('France', 'FRA', 7, 'Silver', 461),
 ('Australia', 'AUS', 8, 'Bronze', 413),
 ('Australia', 'AUS', 8, 'Gold', 293),
 ('Australia', 'AUS', 8, 'Silver', 369),
 ('Denmark', 'DEN', 9, 'Bronze', 152),
 ('De

In [47]:
  country_dic ={}

In [55]:
for result in results:
    if result[2] in country_dic:
        country_dic[result[2]]['medals'][result[3]] = result[4]
    else:
        country ={}

        country['name']  = result[0]
        country['code'] = result[1]
        country['medals'] = {}
        country['medals'][result[3]] = result[4]
        country_dic[result[2]] = country

In [54]:
country_dic[2]
  

{'name': 'Austria', 'code': 'AUT', 'medals': {'Silver': 81}}

In [56]:
country_dic

{1: {'name': 'Hungary',
  'code': 'HUN',
  'medals': {'Silver': 308, 'Bronze': 345, 'Gold': 400}},
 2: {'name': 'Austria',
  'code': 'AUT',
  'medals': {'Silver': 81, 'Bronze': 44, 'Gold': 21}},
 3: {'name': 'Greece',
  'code': 'GRE',
  'medals': {'Silver': 62, 'Bronze': 49, 'Gold': 34}},
 4: {'name': 'United States',
  'code': 'USA',
  'medals': {'Silver': 1195, 'Bronze': 1052, 'Gold': 2088}},
 5: {'name': 'Germany',
  'code': 'GER',
  'medals': {'Silver': 350, 'Bronze': 454, 'Gold': 407}},
 6: {'name': 'United Kingdom',
  'code': 'GBR',
  'medals': {'Silver': 591, 'Bronze': 505, 'Gold': 498}},
 7: {'name': 'France',
  'code': 'FRA',
  'medals': {'Silver': 461, 'Bronze': 475, 'Gold': 378}},
 8: {'name': 'Australia',
  'code': 'AUS',
  'medals': {'Silver': 369, 'Bronze': 413, 'Gold': 293}},
 9: {'name': 'Denmark',
  'code': 'DEN',
  'medals': {'Silver': 192, 'Bronze': 152, 'Gold': 147}},
 10: {'name': 'Switzerland',
  'code': 'SUI',
  'medals': {'Silver': 165, 'Bronze': 138, 'Gold': 73